In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier as SklearnDecisionTreeClassifier

from decision_trees import ID3, C45, CART, print_tree

# Configurações de visualização
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

In [13]:
df_tennis = pd.read_csv('JogarTênis.csv', encoding='utf-8')

# Limpeza e padronização (ajustado para o formato do seu arquivo)
df_tennis.columns = [col.strip().capitalize() for col in df_tennis.columns]
df_tennis.rename(columns={'Windy': 'Wind', 'Play': 'Play'}, inplace=True)
df_tennis['Wind'] = df_tennis['Wind'].astype(str).str.strip().replace({'FALSE': 'Weak', 'TRUE': 'Strong'})
df_tennis['Play'] = df_tennis['Play'].astype(str).str.strip().replace({'no': 'No', 'yes': 'Yes'})

print("Dataset Play Tennis:")
display(df_tennis)

X_tennis = df_tennis.drop('Play', axis=1)
y_tennis = df_tennis['Play']

Dataset Play Tennis:


,Outlook,Temperature,Humidity,Wind,Play
0,sunny,hot,high,False,No
1,sunny,hot,high,True,No
2,overcast,hot,high,False,Yes
3,rainy,mild,high,False,Yes
4,rainy,cool,normal,False,Yes
5,rainy,cool,normal,True,No
6,overcast,cool,normal,True,Yes
7,sunny,mild,high,False,No
8,sunny,cool,normal,False,Yes
9,rainy,mild,normal,False,Yes


In [14]:
df_titanic_train = pd.read_csv('train.csv')

# Limpeza
df_titanic_processed = df_titanic_train[['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']].copy()
age_median = df_titanic_processed['Age'].median()
embarked_mode = df_titanic_processed['Embarked'].mode()[0]

# Usando a forma recomendada para evitar o FutureWarning
df_titanic_processed['Age'] = df_titanic_processed['Age'].fillna(age_median)
df_titanic_processed['Embarked'] = df_titanic_processed['Embarked'].fillna(embarked_mode)

# Partição
X_titanic = df_titanic_processed.drop('Survived', axis=1)
y_titanic = df_titanic_processed['Survived']
X_train, X_test, y_train, y_test = train_test_split(
    X_titanic, y_titanic, test_size=0.2, random_state=42, stratify=y_titanic
)

print("Dados do Titanic preparados e divididos")
display(X_train.head())

Dados do Titanic preparados e divididos


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
692,3,male,28.0,0,0,56.4958,S
481,2,male,28.0,0,0,0.0000,S
527,1,male,28.0,0,0,221.7792,S
855,3,female,18.0,0,1,9.3500,S
801,2,female,31.0,1,1,26.2500,S


In [15]:
print("--- Validação no Dataset Play Tennis ---")

# Treinando ID3
id3_tennis = ID3(max_depth=3)
id3_tennis.fit(X_tennis, y_tennis)
print("\nÁrvore de Decisão gerada pelo ID3:")
print_tree(id3_tennis, id3_tennis.root)

# Treinando C4.5
c45_tennis = C45(max_depth=3)
c45_tennis.fit(X_tennis, y_tennis)
print("\nÁrvore de Decisão gerada pelo C4.5:")
print_tree(c45_tennis, c45_tennis.root)

--- Validação no Dataset Play Tennis ---

Árvore de Decisão gerada pelo ID3:
Feature: Outlook (Impureza: 0.94, Amostras: 14)
|   ├── Se = 'overcast': Prediz: Yes (Impureza: -0.00, Amostras: 4)
|   ├── Se = 'rainy': Feature: Wind (Impureza: 0.97, Amostras: 5)
|   |   ├── Se = 'False': Prediz: Yes (Impureza: -0.00, Amostras: 3)
|   |   ├── Se = 'True': Prediz: No (Impureza: -0.00, Amostras: 2)
|   ├── Se = 'sunny': Feature: Humidity (Impureza: 0.97, Amostras: 5)
|   |   ├── Se = 'high': Prediz: No (Impureza: -0.00, Amostras: 3)
|   |   ├── Se = 'normal': Prediz: Yes (Impureza: -0.00, Amostras: 2)

Árvore de Decisão gerada pelo C4.5:
Feature: Outlook (Impureza: 0.94, Amostras: 14)
|   ├── Se = 'overcast': Prediz: Yes (Impureza: -0.00, Amostras: 4)
|   ├── Se = 'rainy': Feature: Wind (Impureza: 0.97, Amostras: 5)
|   |   ├── Se = 'False': Prediz: Yes (Impureza: -0.00, Amostras: 3)
|   |   ├── Se = 'True': Prediz: No (Impureza: -0.00, Amostras: 2)
|   ├── Se = 'sunny': Feature: Humidity (Im

In [16]:
print("\n--- Experimento: ID3 no Titanic ---")

# Criando cópias para não alterar os dataframes originais
X_train_id3 = X_train.copy()
X_test_id3 = X_test.copy()

# 1. Discretizar 'Age' no treino, tratando duplicatas
age_qcut_result = pd.qcut(X_train_id3['Age'], q=4, labels=['Criança', 'Jovem', 'Adulto', 'Idoso'], retbins=True, duplicates='drop')
X_train_id3['Age'] = age_qcut_result[0]
age_bins = age_qcut_result[1]

# 2. Discretizar 'Fare' no treino, tratando duplicatas
fare_qcut_result = pd.qcut(X_train_id3['Fare'], q=4, labels=['Baixo', 'Médio', 'Alto', 'Muito Alto'], retbins=True, duplicates='drop')
X_train_id3['Fare'] = fare_qcut_result[0]
fare_bins = fare_qcut_result[1]

# 3. Aplicar os mesmos limites (bins) no conjunto de teste
X_test_id3['Age'] = pd.cut(X_test_id3['Age'], bins=age_bins, labels=['Criança', 'Jovem', 'Adulto', 'Idoso'], include_lowest=True)
X_test_id3['Fare'] = pd.cut(X_test_id3['Fare'], bins=fare_bins, labels=['Baixo', 'Médio', 'Alto', 'Muito Alto'], include_lowest=True)

# 4. Lidar com possíveis valores nulos no teste
X_test_id3['Age'] = X_test_id3['Age'].fillna('Adulto')
X_test_id3['Fare'] = X_test_id3['Fare'].fillna('Baixo')


print("Discretização para ID3 concluída com sucesso.")
display(X_train_id3.head())

# Treinando o modelo
id3_titanic = ID3(max_depth=4)
id3_titanic.fit(X_train_id3, y_train)
print("\nÁrvore de Decisão gerada pelo ID3 (Titanic):")
print_tree(id3_titanic, id3_titanic.root)

# Avaliação
y_pred_id3 = id3_titanic.predict(X_test_id3)
print("\n--- Métricas de Desempenho (ID3) ---")
print(f"Acurácia: {accuracy_score(y_test, y_pred_id3):.4f}")
print(classification_report(y_test, y_pred_id3, zero_division=0))


--- Experimento: ID3 no Titanic ---
Discretização para ID3 concluída com sucesso.


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
692,3,male,Jovem,0,0,Muito Alto,S
481,2,male,Jovem,0,0,Baixo,S
527,1,male,Jovem,0,0,Muito Alto,S
855,3,female,Criança,0,1,Médio,S
801,2,female,Adulto,1,1,Alto,S



Árvore de Decisão gerada pelo ID3 (Titanic):
Feature: Sex (Impureza: 0.96, Amostras: 712)
|   ├── Se = 'female': Feature: Fare (Impureza: 0.82, Amostras: 253)
|   |   ├── Se = 'Alto': Feature: Age (Impureza: 0.90, Amostras: 72)
|   |   |   ├── Se = 'Adulto': Feature: Embarked (Impureza: 0.78, Amostras: 13)
|   |   |   |   ├── Se = 'C': Prediz: 1 (Impureza: -0.00, Amostras: 2)
|   |   |   |   ├── Se = 'Q': Prediz: 0 (Impureza: -0.00, Amostras: 1)
|   |   |   |   ├── Se = 'S': Prediz: 1 (Impureza: 0.72, Amostras: 10)
|   |   |   ├── Se = 'Criança': Feature: Embarked (Impureza: 0.90, Amostras: 22)
|   |   |   |   ├── Se = 'C': Prediz: 1 (Impureza: 0.97, Amostras: 5)
|   |   |   |   ├── Se = 'S': Prediz: 1 (Impureza: 0.87, Amostras: 17)
|   |   |   ├── Se = 'Idoso': Feature: Embarked (Impureza: 0.95, Amostras: 16)
|   |   |   |   ├── Se = 'C': Prediz: 1 (Impureza: 1.00, Amostras: 2)
|   |   |   |   ├── Se = 'Q': Prediz: 0 (Impureza: -0.00, Amostras: 1)
|   |   |   |   ├── Se = 'S': Prediz

In [17]:
print("\n--- Experimento: C4.5 no Titanic ---")

# Treinando o modelo
c45_titanic = C45(max_depth=4)
c45_titanic.fit(X_train, y_train)
print("\nÁrvore de Decisão gerada pelo C4.5 (Titanic):")
print_tree(c45_titanic, c45_titanic.root)

# Avaliação
y_pred_c45 = c45_titanic.predict(X_test)
print("\n--- Métricas de Desempenho (C4.5) ---")
print(f"Acurácia: {accuracy_score(y_test, y_pred_c45):.4f}")
print(classification_report(y_test, y_pred_c45))


--- Experimento: C4.5 no Titanic ---

Árvore de Decisão gerada pelo C4.5 (Titanic):
Feature: Sex (Impureza: 0.96, Amostras: 712)
|   ├── Se = 'female': Feature: SibSp (Impureza: 0.82, Amostras: 253)
|   |   ├── <= 6.00: Feature: Parch (Impureza: 0.81, Amostras: 251)
|   |   |   ├── <= 5.50: Feature: Fare (Impureza: 0.81, Amostras: 250)
|   |   |   |   ├── <= 6.99: Prediz: 0 (Impureza: -0.00, Amostras: 1)
|   |   |   |   ├── > 6.99: Prediz: 1 (Impureza: 0.80, Amostras: 249)
|   |   |   ├── > 5.50: Prediz: 0 (Impureza: -0.00, Amostras: 1)
|   |   ├── > 6.00: Prediz: 0 (Impureza: -0.00, Amostras: 2)
|   ├── Se = 'male': Feature: Age (Impureza: 0.69, Amostras: 459)
|   |   ├── <= 0.96: Prediz: 1 (Impureza: -0.00, Amostras: 5)
|   |   ├── > 0.96: Feature: Age (Impureza: 0.67, Amostras: 454)
|   |   |   ├── <= 77.00: Feature: Fare (Impureza: 0.67, Amostras: 453)
|   |   |   |   ├── <= 387.66: Prediz: 0 (Impureza: 0.66, Amostras: 452)
|   |   |   |   ├── > 387.66: Prediz: 1 (Impureza: -0.00,

In [18]:
print("\n--- Experimento: CART no Titanic ---")

# Treinando o nosso CART
cart_titanic = CART(max_depth=4)
cart_titanic.fit(X_train, y_train)
print("\nÁrvore de Decisão gerada pelo nosso CART (Titanic):")
print_tree(cart_titanic, cart_titanic.root)

# Avaliação do nosso CART
y_pred_cart = cart_titanic.predict(X_test)
print("\n--- Métricas de Desempenho (Nosso CART) ---")
print(f"Acurácia: {accuracy_score(y_test, y_pred_cart):.4f}")
print(classification_report(y_test, y_pred_cart))


# Baseline: Scikit-learn CART
print("\n--- Baseline: Scikit-learn CART ---")
# Scikit-learn precisa de dados numéricos
X_train_sklearn = X_train.copy()
X_test_sklearn = X_test.copy()
for col in ['Sex', 'Embarked']:
    X_train_sklearn[col] = X_train_sklearn[col].astype('category').cat.codes
    X_test_sklearn[col] = X_test_sklearn[col].astype('category').cat.codes

sklearn_cart = SklearnDecisionTreeClassifier(criterion='gini', max_depth=4, random_state=42)
sklearn_cart.fit(X_train_sklearn, y_train)
y_pred_sklearn = sklearn_cart.predict(X_test_sklearn)

print("\n--- Métricas de Desempenho (Scikit-learn CART) ---")
print(f"Acurácia: {accuracy_score(y_test, y_pred_sklearn):.4f}")
print(classification_report(y_test, y_pred_sklearn))


--- Experimento: CART no Titanic ---

Árvore de Decisão gerada pelo nosso CART (Titanic):
Feature: Sex (Impureza: 0.47, Amostras: 712)
|   ├── in ('female',): Feature: Pclass (Impureza: 0.38, Amostras: 253)
|   |   ├── <= 2.50: Feature: Fare (Impureza: 0.10, Amostras: 137)
|   |   |   ├── <= 28.86: Feature: Age (Impureza: 0.20, Amostras: 54)
|   |   |   |   ├── <= 56.00: Prediz: 1 (Impureza: 0.17, Amostras: 53)
|   |   |   |   ├── > 56.00: Prediz: 0 (Impureza: 0.00, Amostras: 1)
|   |   |   ├── > 28.86: Feature: Parch (Impureza: 0.02, Amostras: 83)
|   |   |   |   ├── <= 1.50: Prediz: 1 (Impureza: 0.00, Amostras: 71)
|   |   |   |   ├── > 1.50: Prediz: 1 (Impureza: 0.15, Amostras: 12)
|   |   ├── > 2.50: Feature: Embarked (Impureza: 0.50, Amostras: 116)
|   |   |   ├── in ('S',): Feature: Fare (Impureza: 0.46, Amostras: 71)
|   |   |   |   ├── <= 17.25: Prediz: 1 (Impureza: 0.50, Amostras: 44)
|   |   |   |   ├── > 17.25: Prediz: 0 (Impureza: 0.25, Amostras: 27)
|   |   |   ├── not in